In [3]:
%pip install loguru

  Using cached loguru-0.7.3-py3-none-any.whl.metadata (22 kB)
  Using cached win32_setctime-1.2.0-py3-none-any.whl.metadata (2.4 kB)
Using cached loguru-0.7.3-py3-none-any.whl (61 kB)
Using cached win32_setctime-1.2.0-py3-none-any.whl (4.1 kB)
Note: you may need to restart the kernel to use updated packages.


# ours 먼저 돌리기

In [1]:
import json
import os
import time
import win32com.client
import pythoncom
import jsonlines
from pathlib import Path
from main_original import main  # main 함수 import

def load_instructions(json_path):
    """instruction JSON 파일을 로드합니다."""
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except Exception as e:
        print(f"JSON 파일 로드 오류: {e}")
        return {}

def get_slide_number(instruction_key):
    """instruction key에서 슬라이드 번호를 추출합니다."""
    if '-' in instruction_key:
        return instruction_key.split('-')[0]
    return instruction_key

def write_log(log_file, log_data):
    """로그 파일에 안전하게 데이터를 기록합니다."""
    max_retries = 3
    for attempt in range(max_retries):
        try:
            with jsonlines.open(log_file, mode='a') as writer:
                writer.write(log_data)
            return True
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"로그 작성 오류, 재시도 중 ({attempt+1}/{max_retries}): {e}")
                time.sleep(0.5)
            else:
                print(f"로그 작성 최종 실패: {e}")
                return False

def process_instruction(instruction_key, instruction_text, result_folder, log_file):
    """
    주어진 instruction을 처리하고 결과를 저장합니다.
    1. PPT 열고 main() 적용
    2. SaveAs, 실패 시 SaveCopyAs → 원위치 이동
    3. 프레젠테이션 즉시 Close
    4. PowerPoint 애플리케이션 Quit
    5. 로그 기록
    """
    slide_num = get_slide_number(instruction_key)
    slide_path = os.path.abspath(f"evaluation/benchmark_ppts/slide_{slide_num}.pptx")
    result_path = os.path.join(result_folder, f"result_slide_{slide_num}_instruction_{instruction_key}.pptx")

    pythoncom.CoInitialize()
    ppt_app = None
    presentation = None
    start_time = time.time()
    status = "error"
    error_message = None
    result = None

    try:
        # PowerPoint 연결 또는 실행
        try:
            ppt_app = win32com.client.GetActiveObject("PowerPoint.Application")
        except:
            ppt_app = win32com.client.Dispatch("PowerPoint.Application")
        if ppt_app is None:
            raise RuntimeError("PowerPoint 애플리케이션을 시작할 수 없습니다.")

        ppt_app.Visible = True
        presentation = ppt_app.Presentations.Open(slide_path)
        if "{slide_num}" in instruction_text:
            formatted_instruction = instruction_text.replace("{slide_num}", "1")
        else:
            formatted_instruction = instruction_text
        
        #formatted_instruction = instruction_text.replace("{slide_num}", str(int(slide_num) + 1))
        result = main(formatted_instruction, rule_base_apply=False)

        # 디렉토리 및 파일 준비
        os.makedirs(os.path.dirname(result_path), exist_ok=True)
        if os.path.exists(result_path):
            try:
                os.remove(result_path)
            except:
                # 잠금 문제 시 이름 변경
                result_path = os.path.join(
                    result_folder,
                    f"result_slide_{slide_num}_instruction_{instruction_key}_{int(time.time())}.pptx"
                )

        absolute_result_path = os.path.abspath(result_path)
        save_success = False

        # 1차: SaveAs
        try:
            presentation.SaveAs(FileName=absolute_result_path)
            save_success = True
        except Exception as save_err:
            # 2차: SaveCopyAs + 이동
            try:
                temp_name = f"temp_{int(time.time())}_{os.path.basename(result_path)}"
                temp_path = os.path.join(os.path.dirname(absolute_result_path), temp_name)
                presentation.SaveCopyAs(FileName=temp_path)
                # 바로 프레젠테이션 닫기
                presentation.Close()
                presentation = None
                time.sleep(0.5)
                if os.path.exists(temp_path):
                    if os.path.exists(absolute_result_path):
                        os.remove(absolute_result_path)
                    os.rename(temp_path, absolute_result_path)
                    save_success = True
            except Exception as alt_err:
                error_message = f"SaveAs 실패: {save_err}; SaveCopyAs 실패: {alt_err}"

        # 저장 후 즉시 닫기
        if save_success:
            status = "success"
            # presentation이 남아있다면 닫기
            if presentation:
                presentation.Close()
                presentation = None

    except Exception as e:
        error_message = str(e)
        print(f"오류 발생: {instruction_key} – {e}")

    finally:
        # 애플리케이션 종료
        try:
            if ppt_app:
                ppt_app.Quit()
        except Exception as quit_err:
            print(f"PowerPoint Quit 오류: {quit_err}")

        # 로그 기록
        end_time = time.time()
        log_data = {
            "instruction_key": instruction_key,
            "slide_num": slide_num,
            "instruction": instruction_text.replace("{slide_num}", str(int(slide_num) + 1)),
            "execution_time": end_time - start_time,
            "status": status
        }
        if status == "success":
            log_data["result"] = result
        if error_message:
            log_data["error_message"] = error_message
        write_log(log_file, log_data)

        try:
            pythoncom.CoUninitialize()
        except:
            pass

def main_experiment():
    total_start = time.time()
    instruction_path = "expanded_instruction_379.json"
    result_folder = "result"
    log_file = os.path.join(result_folder, "experiment_log.jsonl")
    os.makedirs(result_folder, exist_ok=True)

    instructions = load_instructions(instruction_path)
    if not instructions:
        print("인스트럭션 로드 실패, 종료합니다.")
        return

    processed = 0
    for key, text in instructions.items():
        if "{{slide_number}}" in text:
            text = text.replace("{{slide_number}}", "1")
        process_instruction(key, text, result_folder, log_file)
        processed += 1
        time.sleep(1)

    total_end = time.time()
    summary = {
        "total_processed": processed,
        "total_time": total_end - total_start,
        "avg_time": (total_end - total_start) / processed if processed else 0,
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
    }
    write_log(log_file, summary)
    print("실험 완료:", summary)

if __name__ == "__main__":
    main_experiment()



c:\Users\wjdrb\anaconda3\envs\pptagent\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-09 16:52:22.441 | DEBUG    | gemini_api:send_gemini_request:93 - API key: ***********************************M5D0
2025-05-09 16:52:24.758 | DEBUG    | gemini_api:send_gemini_request:93 - API key: ***********************************oWh0
2025-05-09 16:52:30.099 | DEBUG    | gemini_api:send_gemini_request:93 - API key: ***********************************oWh0
2025-05-09 16:52:53.800 | DEBUG    | gemini_api:send_gemini_request:93 - API key: ***********************************oWh0
2025-05-09 16:53:02.353 | DEBUG    | gemini_api:send_gemini_request:93 - API key: ***********************************M5D0
2025-05-09 16:53:07.497 | DEBUG    | gemini_api:send_gemini_request:93 - API key: ***********************************oWh0


KeyboardInterrupt: 